# Exploratory Data Analysis - Crime Data Los Angeles

This notebook is used to explore the questions for the Data Scientist Exercise in July 2020.

- How many crimes were reported over the past 5 years?
- List the top five reported crimes for each year for the past 5 years.
- What are the most common MO codes? Have these changed over the past 5 years?
- How else has reported crime changed over time in the City?
- Based on your analysis, please share any changes to services, programs, or policies that the City should consider.

##  Load libraries

In [1]:
# Import libraries
import os, sys, subprocess
import json
import pandas as pd
import numpy as np
from langdetect import detect
import pickle

pd.set_option('display.max_colwidth', -1)

pd.set_option('display.max_rows', 1000)
#pd.set_option('display.max_columns', 500)

In [2]:
# load project config
terminal_call = ! git rev-parse --show-toplevel
repo_path=terminal_call[0]
project_config_path = os.path.join(repo_path,'project_config.json')

with open(project_config_path,'r') as fp: 
    project_config = json.load(fp)

In [3]:
# import custom module to look at trends
module_path = os.path.join(repo_path,project_config['project_module_relative_path'])
sys.path.append(module_path)

import trends
from trends import get_top_trends as gt
from trends import convert

from importlib import reload

# Load in data

In [4]:
# load in data that was collected
df = pickle.load(open("../data/crime_data.pkl", "rb"))

## How many crimes were reported over the past 5 years?

In [5]:
# Check how many crimes were reported over the past 5 years
# Calculate current date and offset by 5 years
(df["date_rptd"] > (pd.datetime.now()- pd.DateOffset(years=5))).sum()

1117864

### A: 1,117,864 crimes were reported in the past 5 years

## List the top five reported crimes for each year for the past 5 years.

In [176]:
reload(trends)

<module 'trends' (namespace)>

In [6]:
# Select the last 5 years as a dataframe
df_5 = df[df["date_rptd"] > (pd.datetime.now()- pd.DateOffset(years=5))]

In [7]:
# Find the top 5 reported crimes for each year
# Crime is listed in the data under 'crm_cd'
# Custom function 'top_trends' groups by column and selects the top 'n' 
top_crimes = gt.top_trends(df_5, column='year', variable='crm_cd', n=5)

In [9]:
# Add in the name of the crimes to dataframe
top_crimes["crime_name"] = convert.get_names(top_crimes, "crm_cd", crime)

NameError: name 'crime' is not defined

In [155]:
test

,year,crm_cd,count,crime
0,2015,624,8853,Battery - misdemeanor
1,2015,510,8573,Stolen Vehicle
2,2015,440,7866,Theft - $950 & under
3,2015,330,7316,Burg from Vehicle
4,2015,354,7297,None
131,2016,510,18353,Stolen Vehicle
132,2016,624,17942,Battery - misdemeanor
133,2016,330,16779,Burg from Vehicle
134,2016,440,14814,Theft - $950 & under
135,2016,310,14558,Burglary


In [145]:
test["crm_cd"].apply(crime_codes.get)

0      Battery - misdemeanor
1      Stolen Vehicle       
2      Theft - $950 & under 
3      Burg from Vehicle    
4      None                 
131    Stolen Vehicle       
132    Battery - misdemeanor
133    Burg from Vehicle    
134    Theft - $950 & under 
135    Burglary             
261    Battery - misdemeanor
262    Stolen Vehicle       
263    Burg from Vehicle    
264    Burglary             
265    Theft - $950 & under 
397    Battery - misdemeanor
398    Burg from Vehicle    
399    Stolen Vehicle       
400    Theft - $950 & under 
401    Burglary             
532    Battery - misdemeanor
533    Burg from Vehicle    
534    Theft - $950 & under 
535    Stolen Vehicle       
536    None                 
668    Stolen Vehicle       
669    Battery - misdemeanor
670    Burg from Vehicle    
671    Burglary             
672    None                 
Name: crm_cd, dtype: object

## What are the most common MO codes? Have these changed over the past 5 years?

In [36]:
# Find the top 10 most common MO codes across all the years (2010-2020)
df_5["mocodes"].value_counts(ascending = False).head(10)

0344         93842
0329         44407
1501         18472
0325         12737
0416         12317
0329 1300    8302 
1822         8234 
0344 1300    6090 
0344 1606    5898 
0329 1307    4808 
Name: mocodes, dtype: int64

In [37]:
# Find the top 10 MO codes for each year
# MO code is listed in the data under 'mocodes'
# Custom function 'top_trends' groups by column and selects the top 'n' 
gt.top_trends(df_5, column='year', variable='mocodes', n=5)

year  mocodes  
2015  0344         12619
      0329         4784 
      1501         2013 
      0325         1632 
      0416         1573 
2016  0344         22719
      0329         9713 
      1501         4215 
      0325         3037 
      0416         2939 
2017  0344         20205
      0329         9811 
      1501         4436 
      0416         2938 
      0325         2737 
2018  0344         17485
      0329         9032 
      1501         3705 
      0416         2375 
      0325         2354 
2019  0344         15849
      0329         8156 
      1501         2984 
      0325         2327 
      0416         1977 
2020  0344         4965 
      0329         2911 
      1501         1119 
      0344 1300    706  
      0329 1300    698  
Name: mocodes, dtype: int64